In [61]:
import pandas as pd
import numpy as np

In [66]:
df = pd.read_csv('data/new-site.csv',sep='\t')

In [68]:
df.head(20)

,site,dwell-time
0,0,2
1,1,48
2,0,15
3,1,16
4,0,38
5,1,169
6,0,571
7,1,38
8,0,233
9,0,146


In [65]:
from scipy.stats import shapiro
# нулевая и альтернативная гипотезы
H0 = 'Распределения среднего чека в обеих группах нормальные'
H1 = 'Распределение среднего чека в одной или обеих группах отлично от нормального'
alpha = 0.05 # уровень значимости
# вычисляем результат теста Шапиро-Уилка для выборок
shapiro_result_a = shapiro(df.loc[(df['site']=='0')]['dwell-time'])
shapiro_result_b = shapiro(df.loc[(df['site']=='1')]['dwell-time'])
print('p-value группы А', round(shapiro_result_a.pvalue, 2))
print('p-value группы B', round(shapiro_result_b.pvalue, 2))
# сравниваем полученное p-value с уровнем значимости
if shapiro_result_a.pvalue < alpha or shapiro_result_b.pvalue < alpha:
    print("Отвергаем нулевую гипотезу. {}".format(H0))
    print("Альтернативная гипотеза. H1: {}".format(H1))
else:
    print("Принимаем нулевую гипотезу. {}".format(H0))

ValueError: Data must be at least length 3.

In [70]:
def adj_len(d_1, d_2, method='mean'):
    
    # Делаем копии списков, чтобы они
    # не изменились.
    data = d_1.copy()
    d_1, d_2 = d_1.copy(), d_2.copy()
    dif = len(d_1) - len(d_2)

    # Если разницы нет, просто сортировка.
    if dif == 0:
        return data.sort()
    
    # Подфункция, реализует выбор метода.
    def apply_method(data, method):
        if method == 'mean':
            return stat.mean(data)
        elif method == 'med':
            return stat.median(data)
        elif method == 'nan':
            return np.nan
    
    # Если второй лист больше первого,
    # заполняет разницу NaN или средним,
    # в зависимости от выбора.
    if dif > 0:
        return sorted(data)
    elif dif < 0:
        while dif < 0:
            # then d_1 > d_2
            data.append(apply_method(data, method))
            dif+=1
        return sorted(data)

In [74]:
df['dwell-time'] = df['dwell-time'].astype(int)

In [81]:
import statistics as stat

spb = list(df[(df['site']==0)]['dwell-time'])
mgd = list(df[(df['site']==1)]['dwell-time'])

print(spb,mgd)

dfm = pd.DataFrame({'A':adj_len(spb, mgd, 'mean'),\
    'B':adj_len(mgd, spb, 'mean')}).astype(int)

[2, 15, 38, 571, 233, 146, 57, 54, 75, 138, 6, 30, 369, 76, 25, 233, 32, 7, 97, 79, 86, 23, 13, 34, 112, 298, 11, 80, 29, 5, 19, 192, 18, 110, 40, 404, 30, 43, 129, 7, 227, 138, 185, 16, 145, 9, 48, 158, 79, 35, 73, 149, 92, 19, 57, 67, 6, 34, 18, 6, 68, 50, 136, 10, 28, 213, 57, 170, 115, 536, 91, 222, 15, 31, 22, 24, 8, 57, 63, 107, 540, 34, 51, 6, 489, 147, 29, 49, 134, 12, 41, 196, 30, 27, 50, 74, 75, 66, 13, 38, 74, 132, 37, 41, 66, 3, 3, 35, 8, 22, 36, 241, 40, 75, 70, 140, 140, 18, 67, 52, 62, 44, 4, 82, 61, 12, 55, 116, 4, 127, 147, 19, 268, 23, 117, 70, 53, 137, 113, 172, 174, 133, 45, 117, 103, 16, 5, 105, 66, 277, 156, 285, 64, 70, 30, 95, 68, 8, 5, 27, 60, 46, 61, 11, 149, 17, 51, 33, 72, 148, 153, 43, 155, 5, 470, 28, 388, 197, 3, 25, 2, 39, 22, 7, 89, 138] [48, 16, 169, 38, 336, 55, 2, 32, 21, 162, 23, 190, 107, 30, 145, 21, 78, 244, 68, 85, 72, 6, 44, 53, 140, 24, 22, 13, 80, 2, 289, 34, 28, 5, 1, 92, 67, 25, 232, 43, 131, 128, 131, 123, 66, 47, 112, 37, 48, 14, 46, 72, 

In [82]:
import scipy.stats

scipy.stats.wilcoxon(dfm['A'],\
    dfm['B'], alternative='two-sided')

from scipy.stats import ranksums
print('Проверяем равенство:')
print(ranksums(dfm['A'],\
    dfm['B']))

print('Проверяем гипотезу Spb < Mgd:')
print(ranksums(dfm['A'],\
    dfm['B'],\
    alternative = 'greater'))

print('Проверяем гипотезу Spb > Mgd:')
print(ranksums(dfm['A'],\
    dfm['B'],\
    alternative = 'less'))

Проверяем равенство:
RanksumsResult(statistic=-2.1755163833132674, pvalue=0.02959144175119794)
Проверяем гипотезу Spb < Mgd:
RanksumsResult(statistic=-2.1755163833132674, pvalue=0.985204279124401)
Проверяем гипотезу Spb > Mgd:
RanksumsResult(statistic=-2.1755163833132674, pvalue=0.01479572087559897)


In [83]:
from scipy.stats import shapiro, normaltest
alpha = 0.05

H0 = 'Данные распределены нормально'
Ha = 'Данные не распределены нормально (мы отвергаем H0)'

_, p = shapiro(dfm['A'])
print('p=%.3f' % p)

# Интерпретация 

if p > alpha:
	print(H0,'в sbp')
else:
	print(Ha,'в sbp')

_, p = shapiro(dfm['B'])
print('p=%.3f' % p)

# Интерпретация 

if p > alpha:
	print(H0,'в mgd')
else:
	print(Ha,'в mgd')

p=0.000
Данные не распределены нормально (мы отвергаем H0) в sbp
p=0.000
Данные не распределены нормально (мы отвергаем H0) в mgd
